In [1]:
from __future__ import annotations

import dotenv


# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv("../.env", override=True)

True

# Run basic example from Langchain Vectorstores Chroma page

[Read more here](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

# Basic Example

In [ ]:
import rich
from goob_ai import debugger
import logging
import shutil
import logging
import os
import sys
from pathlib import Path
from typing import TYPE_CHECKING, Any, List, Tuple

import faiss

from goob_ai.services import (
    answer_question_from_context,
    bm25_retrieval,
    create_question_answer_from_context_chain,
    encode_from_string,
    encode_pdf,
    get_chunk_by_index,
    read_pdf_to_string,
    replace_t_with_space,
    retrieve_context_per_question,
    retrieve_with_context_overlap,
    show_context,
    split_text_to_chunks_with_indices,
    text_wrap,
)
from langchain.vectorstores import FAISS, VectorStore
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSerializable
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from loguru import logger as LOGGER
from rank_bm25 import BM25Okapi

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

from goob_ai.bot_logger import get_logger, global_log_config
from goob_ai.aio_settings import aiosettings, get_rich_console


global_log_config(
    log_level=logging.getLevelName("DEBUG"),
    json=False,
)


Logger set up with log level: 10


<loguru.logger handlers=[(id=1, level=10, sink=stdout)]>

2024-08-30 19:43:08.600 | DEBUG    | ipykernel.kernelbase:execute_request - kernelbase.py:802 | {'header': {'msg_id': 'fb555bec-5c53badf859af9ba488e789a_69691_39', 'msg_type': 'execute_reply', 'username': 'malcolm', 'session': 'fb555bec-5c53badf859af9ba488e789a', 'date': datetime.datetime(2024, 8, 30, 23, 43, 8, 600282, tzinfo=datetime.timezone.utc), 'version': '5.3'}, 'msg_id': 'fb555bec-5c53badf859af9ba488e789a_69691_39', 'msg_type': 'execute_reply', 'parent_header': {'date': datetime.datetime(2024, 8, 30, 23, 43, 3, 789000, tzinfo=tzutc()), 'msg_id': 'f6046358-b9d1-4a74-90d6-81235667730d', 'msg_type': 'execute_request', 'session': '3decad8a-8ce2-476c-958b-dfb30b8b8939', 'username': '663e5da3-8dec-4def-a540-c5ee41224075', 'version': '5.2'}, 'content': {'status': 'ok', 'execution_count': 2, 'user_expressions': {}, 'payload': []}, 'metadata': {'started': datetime.datetime(2024, 8, 30, 23, 43, 3, 791343, tzinfo=datetime.timezone.utc), 'dependencies_met': True, 'engine': '5270c102-2bf0-4

In [ ]:
# import

path = "example_data/Understanding_Climate_Change.pdf"


In [8]:
def get_faiss_vector_store(path_to_pdf: Path, chunk_size: int = 400, chunk_overlap: int = 200) -> Tuple[VectorStore, List[Document], str]:
    content = read_pdf_to_string(f"{path_to_pdf}")
    docs = split_text_to_chunks_with_indices(content, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    # This line creates a Faiss index using the IndexFlatL2 class. The dimension of the index is determined by the length of the embedding generated for the query "hello world".
    index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

    # This line creates an instance of the FAISS vector store, specifying the embedding function (embeddings), the Faiss index (index), an in-memory document store (InMemoryDocstore()), and an empty dictionary to map index IDs to document store IDs.
    #FIXME: note this is different from the committed code, looks like we are passing multiple arguments that are not present in the code.

    # DISABLED: # vectorstore = FAISS.from_documents(
    # DISABLED: #     docs, embeddings, index=index, docstore_cls=InMemoryDocstore(), index_to_docstore_id={}
    # DISABLED: # )
    vectorstore = FAISS.from_documents(
        docs, embeddings
    )
    return vectorstore, docs, content

In [9]:
vectorstore, docs, content = get_faiss_vector_store(path)

chunks_query_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [10]:
chunk = get_chunk_by_index(vectorstore, 0)
rich.print(chunk.page_content)

Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past century, human 
activities, particularly the burning of fossil fuels and

# Comparing regular retrival and retrival with context window

In [11]:
# Baseline approach
query = "Explain the role of deforestation and fossil fuels in climate change."
baseline_chunk = chunks_query_retriever.get_relevant_documents(query
    ,
    k=1
)
# Focused context enrichment approach
enriched_chunks = retrieve_with_context_overlap(
    vectorstore,
    chunks_query_retriever,
    query,
    num_neighbors=1,
    chunk_size=400,
    chunk_overlap=200
)

rich.print("Baseline Chunk:")
rich.print(baseline_chunk[0].page_content)
rich.print("\nEnriched Chunks:")
rich.print(enriched_chunks[0])

/Users/malcolm/dev/bossjones/goob_ai/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Baseline Chunk:

ntribute 
to climate change. These forests are vital for regulating the Earth's climate and supporting 
indigenous communities and wildlife. 
Agriculture 
Agriculture contributes to climate change through methane emissions from livestock, rice 
paddies, and the use of synthetic fertilizers. Methane is a potent greenhouse gas with a much 
higher heat-trapping capability than CO2, albeit in smaller

Enriched Chunks:

n. 
Boreal Forests 
Boreal forests, found in the northern regions of North America, Europe, and Asia, also play a 
crucial role in sequestering carbon. Logging and land-use changes in these regions contribute 
to climate change. These forests are vital for regulating the Earth's climate and supporting 
indigenous communities and wildlife. 
Agriculture 
Agriculture contributes to climate change through methane emissions from livestock, rice 
paddies, and the use of synthetic fertilizers. Methane is a potent greenhouse gas with a much 
higher heat-trapping capability than CO2, albeit in smaller quantities. 
Livestock Emissions 
Ruminant animals, such as cows and sheep, produce methane during digestion. Manure 
management practices also contribute to methane and nitrous oxide emissions. Innov